In [1]:
import requests
import csv
import time
import os

In [2]:
def send_traj(index, timestamp, servo_positions):
    url = 'http://swanta.wifi.local.cmu.edu/set-traj'
    r = requests.post(url, data={'index': index, 'timestamp': timestamp, 'positions': str(servo_positions)})
    return r

def set_traj_length(length):
    url = 'http://swanta.wifi.local.cmu.edu/set-traj-len'
    r = requests.post(url, data={'length': length})
    return r

def start():
    url = 'http://swanta.wifi.local.cmu.edu/start'
    r = requests.post(url)
    return r

def stop():
    url = 'http://swanta.wifi.local.cmu.edu/stop'
    r = requests.post(url)
    return r

In [29]:
CSV_FILENAME = "gait_0.350Hz_1.000wn.csv"
TRAJ_SPEED = 0.5

set_traj_length(125)

with open(CSV_FILENAME, 'r') as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    index=0
    for row in list(reader)[::8]:
        timestamp = float(row[0]) / TRAJ_SPEED
        servo_positions = [float(x) for x in row[1:]] * 2
        # for k in range(8, 16):
        #     servo_positions[k] = 90 + (servo_positions[k] - 90) * 0.5
        # print("Sending trajectory index %d with timestamp %f and servo positions %s" % (index, timestamp, str(servo_positions)))
        r = send_traj(index, timestamp, servo_positions)
        index += 1

In [53]:
start()
time.sleep(20)
stop()

<Response [200]>

In [30]:
start()

<Response [200]>

In [31]:
stop()

<Response [200]>

In [28]:
# Loop through all CSV files in the current directory
files = []

for filename in os.listdir("."):
    if filename.endswith("wn.csv"):
        files.append(filename)

files.sort()

TRAJ_SPEED = 1

for i,f in enumerate(files):
    if i < 8:
        continue
    print(f"Uploading file {i} of {len(files)}:\t{f}", end="\r")

    set_traj_length(125)
    with open(CSV_FILENAME, 'r') as csvfile:
        reader = csv.reader(csvfile, delimiter=',')
        index=0
        for row in list(reader)[::8]:
            timestamp = float(row[0]) / TRAJ_SPEED
            servo_positions = [float(x) for x in row[1:]] * 2
            # print("Sending trajectory index %d with timestamp %f and servo positions %s" % (index, timestamp, str(servo_positions)))
            r = send_traj(index, timestamp, servo_positions)
            index += 1

    for k in range(3, 0, -1):
        print(f"Starting file {i} of {len(files)}:\t{f}\tin {k}  ", end="\r")
        time.sleep(1)
    print(end='\x1b[2K')
    print(f"Running file {i} of {len(files)}:\t{f}\t ", end="")

    start()
    time.sleep(20)
    stop()
    print("Done!")
    time.sleep(1)



KeyboardInterrupt: 